# Overview

This notebook will be mainly used for the IBM Capstone project

In [98]:
import numpy as np
from bs4 import BeautifulSoup
import urllib.request
import requests
import pandas as pd
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [7]:
!conda install -c conda-forge geopy --yes 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

## Part 1: Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M,

In [8]:
url = ('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

source = requests.get(url).text

soup = BeautifulSoup(source, 'lxml')

table = soup.find('table', attrs={"class":"wikitable sortable"})
table_rows = table.find_all('tr')


res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)


df = pd.DataFrame(res, columns=["Postal_Code", "Neighbourhood", "Borough"])



In [9]:
df.head()

,Postal_Code,Neighbourhood,Borough
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Strip out the values "not assigned"

In [11]:
df.drop(df[df['Borough']=='Not assigned'].index, inplace = True)
df.head()

,Postal_Code,Neighbourhood,Borough
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Clean the multiple neighbourhoods in a single borough

In [20]:
# for seeing whether code works...
# df_test = df

In [12]:
df=df['Borough'].str.split(',', expand=True).stack().reset_index(level=0).set_index('level_0')\
.rename(columns={0:'Borough'}).join(df.drop('Borough', axis=1))

df.head()

,Borough,Postal_Code,Neighbourhood
2,Parkwoods,M3A,North York
3,Victoria Village,M4A,North York
4,Regent Park,M5A,Downtown Toronto
4,Harbourfront,M5A,Downtown Toronto
5,Lawrence Manor,M6A,North York


## Clean the 'not assigned' neighbourhoods'

In [13]:
print(df[df['Neighbourhood']=='Not assigned'])

Empty DataFrame
Columns: [Borough, Postal_Code, Neighbourhood]
Index: []


In [14]:
#print(df[df['Neighbourhood']=='Not assigned'])
pd.set_option('display.max_rows', None)
df

,Borough,Postal_Code,Neighbourhood
2,Parkwoods,M3A,North York
3,Victoria Village,M4A,North York
4,Regent Park,M5A,Downtown Toronto
4,Harbourfront,M5A,Downtown Toronto
5,Lawrence Manor,M6A,North York
5,Lawrence Heights,M6A,North York
6,Queen's Park,M7A,Downtown Toronto
6,Ontario Provincial Government,M7A,Downtown Toronto
8,Islington Avenue,M9A,Etobicoke
8,Humber Valley Village,M9A,Etobicoke


Looking at the code result and reviewing the dataframe, appears there are no Neighbourhoods with the value "not assigned"

## Dataframe shape

In [15]:
df.shape

(217, 3)

## Part 2: Get Foursquare location data for neighbourhoods

In [16]:
filename = "https://cocl.us/Geospatial_data"
headers = ["Postal_Code","Latitude","Longitude"]
df2 = pd.read_csv(filename, names = headers)
df2.head()

,Postal_Code,Latitude,Longitude
0,Postal Code,Latitude,Longitude
1,M1B,43.8066863,-79.1943534
2,M1C,43.7845351,-79.1604971
3,M1E,43.7635726,-79.1887115
4,M1G,43.7709921,-79.2169174


In [17]:
df3 = pd.merge(df, df2, how='left', on='Postal_Code')
df3.head(12)

,Borough,Postal_Code,Neighbourhood,Latitude,Longitude
0,Parkwoods,M3A,North York,43.7532586,-79.3296565
1,Victoria Village,M4A,North York,43.7258823,-79.3155716
2,Regent Park,M5A,Downtown Toronto,43.6542599,-79.3606359
3,Harbourfront,M5A,Downtown Toronto,43.6542599,-79.3606359
4,Lawrence Manor,M6A,North York,43.718518,-79.4647633
5,Lawrence Heights,M6A,North York,43.718518,-79.4647633
6,Queen's Park,M7A,Downtown Toronto,43.6623015,-79.3894938
7,Ontario Provincial Government,M7A,Downtown Toronto,43.6623015,-79.3894938
8,Islington Avenue,M9A,Etobicoke,43.6678556,-79.5322424
9,Humber Valley Village,M9A,Etobicoke,43.6678556,-79.5322424


In [18]:
df3.shape

(217, 5)

In [34]:
df3["Latitude"]=pd.to_numeric(df3["Latitude"])
df3["Longitude"]=pd.to_numeric(df3["Longitude"])

## Part 3: Explore and cluster the neighbourhoods in Toronto

### a: explore the neighbourhood data (basically repeating coursework code for exploring NY, but replacing for Toronto, plus neighbourhood with borough data as only 4 "Neighbourhoods" in Toronto data, so clustering wouldn't make too much sense

In [35]:
df4 = df3[df3['Neighbourhood'].str.contains("Toronto")]
df4.head(78)

,Borough,Postal_Code,Neighbourhood,Latitude,Longitude
2,Regent Park,M5A,Downtown Toronto,43.654260,-79.360636
3,Harbourfront,M5A,Downtown Toronto,43.654260,-79.360636
6,Queen's Park,M7A,Downtown Toronto,43.662301,-79.389494
7,Ontario Provincial Government,M7A,Downtown Toronto,43.662301,-79.389494
15,Garden District,M5B,Downtown Toronto,43.657162,-79.378937
16,Ryerson,M5B,Downtown Toronto,43.657162,-79.378937
28,St. James Town,M5C,Downtown Toronto,43.651494,-79.375418
37,The Beaches,M4E,East Toronto,43.676357,-79.293031
38,Berczy Park,M5E,Downtown Toronto,43.644771,-79.373306
42,Central Bay Street,M5G,Downtown Toronto,43.657952,-79.387383


In [41]:
df4.shape

(78, 5)

In [25]:
from geopy.geocoders import Nominatim

# Get main coordinates of Toronto
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [23]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

    altair:  4.1.0-py_1 conda-forge
    branca:  0.4.1-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


vincent-0.4.4        | 28 KB     | ##################################### | 1

In [38]:
# create map of Toronto using latitude and longitude values
#map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

#map_toronto

In [37]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df4['Latitude'], df4['Longitude'], df4['Borough'], df4['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Observation: Can see distribution of neighbourhoods across Toronto, but densely packed neighbourhoods toward the centre of the city

In [107]:
#my foursquare credentials (hidden for publishing)
CLIENT_ID = '#####' # your Foursquare ID
CLIENT_SECRET = '#####' # your Foursquare Secret
VERSION = '20200524' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: #####
CLIENT_SECRET:#####


Explore sample neighbourhood 

In [42]:
df4 = df4.reset_index(drop=True)

In [44]:
df4.loc[0, 'Neighbourhood']

'Downtown Toronto'

In [46]:
neighbourhood_latitude = df4.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = df4.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = df4.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Downtown Toronto are 43.6542599, -79.3606359.


In [50]:
# limit of number of venues returned by Foursquare API
LIMIT = 100 

# define radius
radius = 500 

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=NUPPQGHUAVAMDYQ5RLGZW0WWDNPMUY1NTLS0HJVD0B4YEGJB&client_secret=XYTL44QJCFF3FIG4MQQTW3OSWW0O0O2W2QBBSZ3R4UYIMABM&v=20200524&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [51]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f0db82295079052bcdbd97c'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 44,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

In [52]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [79]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough',
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [80]:
toronto_venues = getNearbyVenues(names=df4['Borough'],
                                   latitudes=df4['Latitude'],
                                   longitudes=df4['Longitude']
                                  )

Regent Park
 Harbourfront
Queen's Park
 Ontario Provincial Government
Garden District
 Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond
 Adelaide
 King
Dufferin
 Dovercourt Village
Harbourfront East
 Union Station
 Toronto Islands
Little Portugal
 Trinity
The Danforth West
 Riverdale
Toronto Dominion Centre
 Design Exchange
Brockton
 Parkdale Village
 Exhibition Place
India Bazaar
 The Beaches West
Commerce Court
 Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
 Forest Hill Road Park
High Park
 The Junction South
North Toronto West
  Lawrence Park
The Annex
 North Midtown
 Yorkville
Parkdale
 Roncesvalles
Davisville
University of Toronto
 Harbord
Runnymede
 Swansea
Moore Park
 Summerhill East
Kensington Market
 Chinatown
 Grange Park
Summerhill West
 Rathnelly
 South Hill
 Forest Hill SE
 Deer Park
CN Tower
 King and Spadina
 Railway Lands
 Harbourfront West
 Bathurst Quay
 South Niagara
 Island airport


In [81]:
print(toronto_venues.shape)
toronto_venues.head()

(3185, 7)


,Borough,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Regent Park,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Regent Park,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,Regent Park,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Regent Park,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [82]:
toronto_venues.groupby('Borough').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Lawrence Park,21,21,21,21,21,21
Adelaide,94,94,94,94,94,94
Bathurst Quay,17,17,17,17,17,17
Cabbagetown,43,43,43,43,43,43
Chinatown,63,63,63,63,63,63
Deer Park,17,17,17,17,17,17
Design Exchange,100,100,100,100,100,100
Dovercourt Village,14,14,14,14,14,14
Exhibition Place,23,23,23,23,23,23


In [83]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 230 unique categories.


In [85]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Borough'] = toronto_venues['Borough'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Borough,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Regent Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Regent Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Regent Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Regent Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Regent Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
toronto_onehot.shape

(3185, 231)

In [87]:
toronto_grouped = toronto_onehot.groupby('Borough').mean().reset_index()
toronto_grouped

,Borough,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Lawrence Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619
1,Adelaide,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021277,0.000000,0.00,...,0.000000,0.000000,0.00000,0.00,0.010638,0.000000,0.000000,0.000000,0.010638,0.000000
2,Bathurst Quay,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Cabbagetown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Chinatown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00000,0.00,0.047619,0.000000,0.047619,0.015873,0.000000,0.000000
5,Deer Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.00,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000
6,Design Exchange,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.00,...,0.000000,0.000000,0.00000,0.01,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000
7,Dovercourt Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Exhibition Place,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Forest Hill Road Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.25000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [88]:
toronto_grouped.shape

(77, 231)

In [89]:
num_top_venues = 5

for hood in toronto_grouped['Borough']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----  Lawrence Park----
                 venue  freq
0       Clothing Store  0.14
1  Sporting Goods Shop  0.10
2          Coffee Shop  0.10
3          Yoga Studio  0.05
4       Cosmetics Shop  0.05


---- Adelaide----
         venue  freq
0  Coffee Shop  0.09
1         Café  0.05
2        Hotel  0.04
3   Restaurant  0.04
4          Gym  0.03


---- Bathurst Quay----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3           Airport  0.06
4             Plane  0.06


---- Cabbagetown----
                venue  freq
0         Coffee Shop  0.07
1         Pizza Place  0.07
2          Restaurant  0.07
3  Chinese Restaurant  0.05
4              Bakery  0.05


---- Chinatown----
                           venue  freq
0                           Café  0.08
1                    Coffee Shop  0.06
2                            Bar  0.05
3          Vietnamese Restaurant  0.05
4  Vegetarian / Vegan Restaurant  0.05


---- Deer Park----
        

           venue  freq
0  Grocery Store  0.24
1           Café  0.18
2           Park  0.12
3     Baby Store  0.06
4          Diner  0.06


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.09
1  Japanese Restaurant  0.06
2     Sushi Restaurant  0.06
3           Restaurant  0.04
4              Gay Bar  0.04


----Commerce Court----
         venue  freq
0  Coffee Shop  0.12
1   Restaurant  0.07
2         Café  0.07
3        Hotel  0.06
4          Gym  0.04


----Davisville----
            venue  freq
0     Pizza Place  0.11
1    Dessert Shop  0.08
2  Sandwich Place  0.08
3             Gym  0.05
4     Coffee Shop  0.05


----Davisville North----
                  venue  freq
0  Gym / Fitness Center  0.14
1        Sandwich Place  0.14
2        Breakfast Spot  0.14
3                  Park  0.14
4     Food & Drink Shop  0.14


----Dufferin----
                       venue  freq
0                   Pharmacy  0.14
1                     Bakery  0.14
2         

In [90]:
# function to sort venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [91]:
# top 10 venues for each borough

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Borough'] = toronto_grouped['Borough']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Lawrence Park,Clothing Store,Sporting Goods Shop,Coffee Shop,Yoga Studio,Rental Car Location,Café,Mexican Restaurant,Fast Food Restaurant,Salon / Barbershop,Cosmetics Shop
1,Adelaide,Coffee Shop,Café,Hotel,Restaurant,Deli / Bodega,Thai Restaurant,Gym,Salad Place,Sushi Restaurant,Cosmetics Shop
2,Bathurst Quay,Airport Service,Airport Lounge,Airport Terminal,Airport,Harbor / Marina,Coffee Shop,Plane,Rental Car Location,Sculpture Garden,Boat or Ferry
3,Cabbagetown,Restaurant,Coffee Shop,Pizza Place,Italian Restaurant,Café,Pub,Bakery,Chinese Restaurant,Grocery Store,Pharmacy
4,Chinatown,Café,Coffee Shop,Bar,Mexican Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Dessert Shop,Pizza Place,Park,Bakery


### b: cluster the neighbourhood data

In [100]:
# set number of clusters
kclusters = 7

toronto_grouped_clustering = toronto_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 1, 0, 0, 6, 0, 0, 0, 3], dtype=int32)

In [104]:
# add clustering labels

#for repeating exercise with different k values, run below line
del neighbourhoods_venues_sorted['Cluster Labels']

neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df4

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Borough'), on='Borough')

toronto_merged.head() # check the last columns!

,Borough,Postal_Code,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Regent Park,M5A,Downtown Toronto,43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Breakfast Spot,Café,Pub,Theater,Yoga Studio,Mexican Restaurant,Shoe Store
1,Harbourfront,M5A,Downtown Toronto,43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Breakfast Spot,Café,Pub,Theater,Yoga Studio,Mexican Restaurant,Shoe Store
2,Queen's Park,M7A,Downtown Toronto,43.662301,-79.389494,0,Coffee Shop,Diner,Yoga Studio,College Cafeteria,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Park
3,Ontario Provincial Government,M7A,Downtown Toronto,43.662301,-79.389494,0,Coffee Shop,Diner,Yoga Studio,College Cafeteria,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Park
4,Garden District,M5B,Downtown Toronto,43.657162,-79.378937,0,Clothing Store,Coffee Shop,Hotel,Cosmetics Shop,Café,Bubble Tea Shop,Japanese Restaurant,Italian Restaurant,Fast Food Restaurant,Lingerie Store


In [105]:
#check the entire table
toronto_merged

,Borough,Postal_Code,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Regent Park,M5A,Downtown Toronto,43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Breakfast Spot,Café,Pub,Theater,Yoga Studio,Mexican Restaurant,Shoe Store
1,Harbourfront,M5A,Downtown Toronto,43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Breakfast Spot,Café,Pub,Theater,Yoga Studio,Mexican Restaurant,Shoe Store
2,Queen's Park,M7A,Downtown Toronto,43.662301,-79.389494,0,Coffee Shop,Diner,Yoga Studio,College Cafeteria,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Park
3,Ontario Provincial Government,M7A,Downtown Toronto,43.662301,-79.389494,0,Coffee Shop,Diner,Yoga Studio,College Cafeteria,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Park
4,Garden District,M5B,Downtown Toronto,43.657162,-79.378937,0,Clothing Store,Coffee Shop,Hotel,Cosmetics Shop,Café,Bubble Tea Shop,Japanese Restaurant,Italian Restaurant,Fast Food Restaurant,Lingerie Store
5,Ryerson,M5B,Downtown Toronto,43.657162,-79.378937,0,Clothing Store,Coffee Shop,Hotel,Cosmetics Shop,Café,Bubble Tea Shop,Japanese Restaurant,Italian Restaurant,Fast Food Restaurant,Lingerie Store
6,St. James Town,M5C,Downtown Toronto,43.651494,-79.375418,0,Coffee Shop,Café,Restaurant,Italian Restaurant,Cosmetics Shop,American Restaurant,Bakery,Cocktail Bar,Clothing Store,Japanese Restaurant
7,The Beaches,M4E,East Toronto,43.676357,-79.293031,6,Health Food Store,Neighborhood,Trail,Pub,Distribution Center,Department Store,Dessert Shop,Diner,Discount Store,Yoga Studio
8,Berczy Park,M5E,Downtown Toronto,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Café,Cheese Shop,Farmers Market,Seafood Restaurant,Bakery,Restaurant,Beer Bar,Lounge
9,Central Bay Street,M5G,Downtown Toronto,43.657952,-79.387383,0,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Thai Restaurant,Bubble Tea Shop,Japanese Restaurant,Bar,Burger Joint,Salad Place


In [106]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Borough'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Observation: Clustering is interesting... Blue boroughs are separated by reds, most boroughs are red, and very few orange / purple / green. For proper exercise, would experiment with clustering logic. I've started with 5 and replaced by 7 above, to see what it looks like or the difference it makes. Theme is the same.